<a href="https://colab.research.google.com/github/AnuttharadeSilva/t_analyzer_model/blob/master/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from string import punctuation 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
df = pd.read_csv('./drive/My Drive/tweet/tweets.csv', delimiter = ',',encoding='ISO-8859-1')
#df2 = pd.read_csv('./drive/My Drive/tweet/reviews.csv', delimiter = ',',encoding='ISO-8859-1')
#frames = [df1,df2]
#df = pd.concat(frames)
#df = pd.DataFrame(df.values, columns = ['Sentiment','SentimentText'])
df = df.dropna()
df = df.reset_index(drop=True)
df_length =len(df)

In [ ]:
df.head(10)

,Sentiment,SentimentText
0,0,I didn't realize it was THAT deep. Geez giv...
1,0,I hate it when any athlete appears to tear ...
2,0,i miss you guys too i think i'm wearing...
3,0,-- Meet your Meat http://bit.ly/15SSCI
4,0,My horsie is moving on Saturday morning.
5,0,No Sat off...Need to work 6 days a week
6,0,Really Dont Like Doing my Room Its So Borin...
7,0,"SOX! Floyd was great, but relievers nee..."
8,0,times by like a million
9,1,uploading pictures on friendster


In [ ]:
corpus = []
stopwords = set(stopwords.words('english')+list(punctuation)+['AT_USER','URL'])
for i in range(0, df_length):    
    tweet = re.sub('[^a-zA-Z]', ' ', df['SentimentText'][i])
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet)
    #tweet = tweet.split()
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    tweet = tokenizer.tokenize(tweet)
    #ps = PorterStemmer()
    lmz = nltk.stem.WordNetLemmatizer()
    tweet = [lmz.lemmatize(word) for word in tweet if not word in stopwords ]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

In [ ]:
corpus[0:10]

['realize deep geez give girl warning atleast',
 'hate athlete appears tear acl live television',
 'miss guy think wearing skinny jean cute sweater heel really sure today',
 'meet meat http bit ly ssci',
 'horsie moving saturday morning',
 'sat need work day week',
 'really dont like room boring sick wardrobe cant waiit till walk one yay',
 'sox floyd great reliever need scolding',
 'time like million',
 'uploading picture friendster']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
cv = CountVectorizer()
td = TfidfTransformer(use_idf=True, norm='l2',smooth_idf=True)
docs = cv.fit_transform(corpus).toarray()
X = td.fit_transform(docs).toarray()
y = df.iloc[:, 0].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)


In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
#accuracy score
from sklearn import metrics
print('Accuracy: ',metrics.accuracy_score(y_test, y_pred))

#confusion matrix
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

#error
classification_error = (FP + FN) / float(TP + TN + FP + FN)
print('Error: ',classification_error)

#recall/sensitivity
print('Sensitivity: ',metrics.recall_score(y_test, y_pred))

#specificity
specificity = TN / (TN + FP)
print('Specificity: ',specificity)

#false positive rate
print('false positive rate: ',1 - specificity)

#precision
print('precision: ',metrics.precision_score(y_test, y_pred))

#f1 score
from sklearn.metrics import f1_score
print('f1 score: ',f1_score(y_test, y_pred))

Accuracy:  0.7132867132867133
[[1706  504]
 [ 685 1252]]
Error:  0.2867132867132867
Sensitivity:  0.6463603510583377
Specificity:  0.7719457013574661
false positive rate:  0.2280542986425339
precision:  0.7129840546697038
f1 score:  0.678039534253994


In [ ]:
# from sklearn.externals import joblib
# joblib.dump(classifier, 'rf_new.pkl')
